Let's import the dataframe and cosine matrix of the Qwen-3 model

In [1]:
import pandas as pd

In [2]:
import gc

In [5]:
Full_cosine_matrix = pd.read_pickle("Full_cosine_matrix_Qwen-3-6B.pkl")

In [6]:
sg_df_clean = pd.read_csv("sg_df_clean.csv")

In [7]:
gc.collect()

0

In [8]:
sg_df_clean["rating_ratio"] = sg_df_clean["rating"]/100
print(sg_df_clean["rating_ratio"])

0        0.352941
1        0.913793
2        1.000000
3        0.862069
4        0.639706
           ...   
72366    0.808989
72367    0.681199
72368    1.000000
72369    1.000000
72370    0.666667
Name: rating_ratio, Length: 72371, dtype: float64


In [9]:
c = sg_df_clean["rating_ratio"].mean()
m= sg_df_clean["user_reviews"].median()

In [10]:
def weighted_game_score(x, c=c, m=m):
    r = x["rating_ratio"] #I am taking the rating_ratio for each game (row) and storing it as variable r
    n = x["user_reviews"]#I am taking the user_reviews for each game (row) and storing it as variable n
    return ((n*r) / (n+m) + (m*c) / (n+m))

In [11]:
sg_df_clean["game_score"] = sg_df_clean.apply(weighted_game_score, axis=1)

In [12]:
sg_df_clean[["name","game_score"]].sample(20)

,name,game_score
61478,Salaryman RESCUE!,0.737490
53364,Shooting Waltz,0.785109
41545,Astrozombies,0.794876
63505,Gas Gas Rocket!,0.844389
41052,The Wizard and The Slug,0.926021
17441,Eruption 爆发,0.844935
68358,Backrooms Security Room,0.774364
59844,Don't Fall,0.795825
16519,Horror Adventure VR,0.671065
16647,NBA Playgrounds,0.708415


In [13]:
def recommend_games(game, df = sg_df_clean, sim_matrix = Full_cosine_matrix):
    # 1. Find the game's index in the dataframe to use it in the similarity matrix
    try: 
        index = df[df['name'] == game].index[0]
    except IndexError:
        return "The game you typed does not exist in the database. Please make sure the spelling exactly matches the game on steam" #to give an error if the spelling is incorrect
    
    # 2. We create a temp dataframe to modify without changing the original
    temp_df = df.copy()
    
    # 3. We create a new column with the list of cosine similarities for the specified game
    temp_df['similarity'] = sim_matrix[index]
    
    # 4. FILTER (Get top 20 matches)
    # Sort by similarity (Descending)
    # iloc[1:21] grabs the top 20, skipping the game itself (which is the first one since it's the most similar)
    top_similar = temp_df.sort_values('similarity', ascending=False).iloc[1:21]
    
    # 4. RANK (Pick top 5 best quality)
    # Sort the 20 candidates by 'game_score' and pick the top 5
    top_picks = top_similar.sort_values('game_score', ascending=False).head(5)
    
    # 5. Return only the relevant columns
    cols = ['name', 'similarity', 'game_score']
    return top_picks[cols]



In [14]:
recommend_games(game = "Hogwarts Legacy")

,name,similarity,game_score
57089,Arto,0.606623,0.880158
41993,Exiled Kingdoms,0.589081,0.874805
41303,Horizon Zero Dawn™ Complete Edition,0.607325,0.861853
38542,Battle for Esturia,0.594730,0.855273
4416,Hazordhu,0.611287,0.838831


In [16]:
recommend_games(game = "The Elder Scrolls V: Skyrim Special Edition")

,name,similarity,game_score
16413,The Elder Scrolls IV: Oblivion® Game of the Ye...,0.569550,0.955539
51296,The Elder Scrolls IV: Oblivion® Game of the Ye...,0.554443,0.955496
5785,The Elder Scrolls III: Morrowind® Game of the ...,0.544408,0.953199
9109,The Elder Scrolls V: Skyrim,0.577546,0.948258
63572,Horizon Forbidden West™ Complete Edition,0.559197,0.938925


In [29]:
def rec_rating():
    a = recommend_games(game = "The Elder Scrolls V: Skyrim Special Edition")
    b = recommend_games(game = "DOOM Eternal")
    c = recommend_games(game = "Hollow Knight")
    d = recommend_games(game = "Hades")
    e = recommend_games(game = "ELDEN RING")
    return a, b, c, d, e


In [35]:
rec_rating()

(                                                    name  similarity  \
 16413  The Elder Scrolls IV: Oblivion® Game of the Ye...    0.569550   
 51296  The Elder Scrolls IV: Oblivion® Game of the Ye...    0.554443   
 5785   The Elder Scrolls III: Morrowind® Game of the ...    0.544408   
 9109                         The Elder Scrolls V: Skyrim    0.577546   
 63572           Horizon Forbidden West™ Complete Edition    0.559197   
 
        game_score  
 16413    0.955539  
 51296    0.955496  
 5785     0.953199  
 9109     0.948258  
 63572    0.938925  ,
                      name  similarity  game_score
 12387                DUSK    0.574517    0.978824
 48612      Shadow Warrior    0.555321    0.925725
 60722       Missiles Away    0.578699    0.895053
 18318  Serious Sam 3: BFE    0.562023    0.884007
 11386    Shadow Warrior 2    0.558709    0.876779,
                               name  similarity  game_score
 58738                Blasphemous 2    0.605772    0.934871
 2014 

In order, these are the scores for whether the recommendations are good or not: 
1) 5/5
2) 5/5
3) 4/5
4) 5/5
5) 3/5

Total = 22/25

The good recommendations are better than the previous two, but the bad recommendations are odd. 